In [1]:
import csv
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import matthews_corrcoef as MC
from sklearn.metrics import ConfusionMatrixDisplay as CMD
import matplotlib.pyplot as plt
from collections import Counter as Cnt
import geopandas as gpd
import pickle
import pandas as pd
import json
import random
import os
from tqdm.notebook import tqdm
import geopy.distance
import shutil
from img2vec_pytorch import Img2Vec
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity


Due to some images being near duplicates, due to being from the same stretch of road we decided to do some distance and image similarity correction on the data.

case1 country exists
case2 country and state exists
case3 country and state and road exists
case4 country and road exists

Split eac


In [2]:
with open('../../file_addresses.json') as json_file:
    data = json.load(json_file)
with open("../../mapillary/train.json") as file:
    metadata = json.load(file)
with open('country_to_region.json') as json_file:
    country_to_region = json.load(json_file)

In [3]:
# remove error 
for file in list(data.keys()):
    if data[file] == "Error":
        data.pop(file)
        # print("pop")

In [89]:
#country to state to road to file dictionary

# case1 country exists
# case2 country and state exists
# case3 country and state and road exists
# case4 country and road exists

CSRF = dict()
count = 0
road_err = 0


for file in tqdm(data.keys()):

    # if file not in all_files:
    #     print(file)
    #     print(data[file])
    #     break
    country = data[file]["country"]
    
    # Use country_to_region dict to check type of region
    state = country_to_region[country]
    # Set the actual region/state e.g "state" for U.S turns to Arizona
    if state in data[file].keys():
        state = data[file][state]
    else:
        state = None
        
    # if road exists in file data save to road
    road = None
    if "road" in data[file].keys():
        road = data[file]["road"]
    
    if road == None and state == None: # Case1
        CSRF.setdefault(country,{"no_road_state":[]})
        CSRF[country]["no_road_state"].append(file)
        
    elif road == None: # Case2
        CSRF.setdefault(country,{"no_road_state":[]})
        CSRF[country].setdefault(state,{"no_road":[]})
        CSRF[country][state]["no_road"].append(file)
        
    elif state == None: # Case4
        CSRF.setdefault(country,{"no_road_state":[]})
        CSRF[country].setdefault(road,{"no_state":[]})
        CSRF[country][road]["no_state"].append(file)
        
    elif state != None and road != None: # Case3
        CSRF.setdefault(country,{"no_road_state":[]})
        CSRF[country].setdefault(state,{"no_road":[]})
        CSRF[country][state].setdefault(road,[])
        CSRF[country][state][road].append(file)
        
    else:
        print("error")


  0%|          | 0/741879 [00:00<?, ?it/s]

In [166]:
#geopy.distance.geodesic(coord, coord_comp).km

file_groups = []
for country in CSRF.keys():
    file_groups.append(CSRF[country]["no_road_state"])
    for state_road in CSRF[country].keys():
        if type(CSRF[country][state_road]) == dict:
            # print("here")
            for finalkey in CSRF[country][state_road].keys():
                file_groups.append(CSRF[country][state_road][finalkey])              

all_files = [file for group in file_groups for file in group]

In [172]:
file_groups.sort(key=len, reverse=True)


In [9]:
# with open(f"../pickles/file_groups.pickle", 'wb') as handle:
#     pickle.dump(file_groups, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open("../pickles/file_groups.pickle", 'rb') as handle:
    file_groups = pickle.load(handle)

# Testing below

In [10]:
accept_list = []
unaccept_list = []
count = 0 
error=0
def splitter(lst,stride,start):
    return lst[start::stride]
file_groups = splitter(file_groups,10,0)
random.shuffle(file_groups)
for group in tqdm(file_groups):
    accept_one = False
    
    for file in tqdm(group, leave=False):
        if accept_one == False:
                accept_one = True
                accept_list.append(file)
        for file2 in group:
            try:
                coord1 = (metadata[file]['lat'],metadata[file]['lon'])
                coord2 = (metadata[file2]['lat'],metadata[file2]['lon'])
            except:
                print("error with file or file2 metadata")
            if file == file2:
                pass
            elif file2 in unaccept_list or file in unaccept_list:
                pass
            elif geopy.distance.geodesic(coord1,coord2).km < 1:
                unaccept_list.append(file2)
            else:
                if file not in accept_list:
                    accept_list.append(file)
                    
                else:
                    error += 1
            
        
            

  0%|          | 0/4254 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [116]:
len(file_groups[42510])

1231

In [138]:
len(group)

7404

In [139]:
group = file_groups[42510]
imagelist = [os.path.join("F:\Mapillary\\train\\", file + ".jpg") for file in group]
dest_fpath = '..\\..\\test_images\\group42530\\'
os.makedirs(os.path.dirname(dest_fpath), exist_ok=True)
for f in tqdm(imagelist):
    shutil.copy(f, dest_fpath)

  0%|          | 0/7404 [00:00<?, ?it/s]

In [149]:
group[0]

'--jFINfGE5rt16nmas_Yrg'

In [146]:
from img2vec_pytorch import Img2Vec
from PIL import Image

# img2vec = Img2Vec(cuda=True)
# file = dest_fpath + group[0] + ".jpg"
# img1 = Image.open(file).convert('RGB')

In [161]:
group = file_groups[42510]
dest_fpath = '..\\..\\test_images\\group42510\\'
print(len(group))

1231


In [163]:
from sklearn.metrics.pairwise import cosine_similarity
group = file_groups[42510][:200]

In [175]:
img2vec = Img2Vec(cuda=True)
accept_one = False
file_src = dest_fpath 
accept_list = []
unaccept_set = set()
for file in tqdm(group):
    file_path = file_src + file + ".jpg"
    if accept_one == False:
            accept_one = True
            accept_list.append(file)
    for file2 in tqdm(group):
        file2_path = file_src + file2 + ".jpg"
        if file == file2:
            continue
        elif file2 in unaccept_set or file in unaccept_set:
            continue
        
        img1 = Image.open(file_path).convert('RGB')
        img2 = Image.open(file2_path).convert('RGB')
        vec1 = img2vec.get_vec(img1, tensor=True).reshape(1, -1)
        vec2 = img2vec.get_vec(img2, tensor=True).reshape(1, -1)
        cos_sim = cosine_similarity(vec1, vec2)[0][0]


        if cos_sim > 0.9:
            unaccept_set.add(file2)
        else:
            if file not in accept_list:
                accept_list.append(file)
    print("unaccept length ", len(unaccept_set))

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
with open(f"accept_list.pickle", 'wb') as handle:
    pickle.dump(accept_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [126]:
len(unaccept_set.intersection(set(accept_list)))

0

In [128]:
len(group)

1231

In [210]:
len(unaccept_list)

16771

In [110]:
lengths = [len(group) for group in file_groups]

In [111]:
lengths.sort(reverse=True)

In [15]:
coord1 = (metadata['0UdmHX8jwJs6045dqtAfKA']['lat'],metadata['0UdmHX8jwJs6045dqtAfKA']['lon'])
coord2 = (metadata['YrE3GCc5Re-m5zqYHca9-w']['lat'],metadata['YrE3GCc5Re-m5zqYHca9-w']['lon'])
%timeit geopy.distance.geodesic(coord1,coord2).km

124 µs ± 601 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
for file in accept_list:
    for file2 in accept_list:
        try:
            coord1 = (metadata[file]['lat'],metadata[file]['lon'])
            coord2 = (metadata[file2]['lat'],metadata[file2]['lon'])
        except:
            print("error with file or file2 metadata")
        if file == file2:
            pass
        elif geopy.distance.geodesic(coord1,coord2).km < 10:
            print("unaccept")
        else:
            pass

In [113]:
max_ =0
idxmax = -1
for idx, group in enumerate(file_groups):
    if len(group) > max_:
        max_ = len(group)
        idxmax=idx

In [114]:
idxmax

42530

In [53]:
len(file_groups[2])

76

## 7000*7000

In [16]:
import time

In [66]:
unaccept_set = set()
def foo(idxmax=2):
    for file in tqdm(file_groups[idxmax]):
        remain = file_groups[idxmax].copy()
        for idx in reversed(range(len(remain))):
            try:
                coord1 = (metadata[file]['lat'],metadata[file]['lon'])
                coord2 = (metadata[remain[idx]]['lat'],metadata[remain[idx]]['lon'])
            except:
                print("error with file or file2 metadata")
            if file == remain[idx]:
                pass
            elif remain[idx] in unaccept_set or file in unaccept_set:
                continue
            elif geopy.distance.great_circle(coord1,coord2).km < 1:
                file_groups[idxmax].pop(idx)
                unaccept_set.add(remain[idx])
            else:
                if file not in accept_list:
                    accept_list.append(file)



In [78]:

def splitter(lst,n,start):
    return lst[start::n]

In [79]:
splitter(test,3,1)

[2, 5]

In [103]:
accept_lists = []
for i in range(20):
    with open(f"../pickles/distcorr/accept_list{i}.pickle", 'rb') as handle:
        accept_lists.append(pickle.load(handle))
unaccept_sets = []
for i in range(20):
    with open(f"../pickles/distcorr/unaccept_set{i}.pickle", 'rb') as handle:
        unaccept_sets.append(pickle.load(handle))

In [98]:
len(data)

2833

In [104]:
accepted_files = [file for group in accept_lists for file in group]
unaccept_files = [file for group in unaccept_sets for file in list(group)]

In [107]:
len(accepted_files)

55751

In [105]:
len(unaccept_files)

686128

In [108]:
dist_dict = {"accepted":accepted_files, "unaccepted": unaccept_files}
with open(f"../pickles/distcorr/dist_corr_dict.pickle", 'wb') as handle:
    pickle.dump(dist_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Coscorr load

In [2]:
accept_lists = []

with open(f"../pickles/coscorr/cossim_accept_listtotal.pickle", 'rb') as handle:
    accept_lists.append(pickle.load(handle))
unaccept_sets = []

with open(f"../pickles/coscorr/cossim_unaccept_settotal.pickle", 'rb') as handle:
    unaccept_sets.append(pickle.load(handle))

In [10]:
len(accept_lists[0])

222701

In [8]:
len(unaccept_sets[0])

519178

# Plot road

In [ ]:
road = 0
count = 0
roadfiles = []
nonroadfiles = []
for file in data.keys():
    try:
        data[file]["road"]
        road += 1
        roadfiles.append(file)
    except:
        count += 1
        nonroadfiles.append(file)
        print(data[file])
        print(file)
        # break
        if count == 5:
            break
        continue
print(road)

In [ ]:
long_list = [metadata[file]["lon"] for file in nonroadfiles]
lat_list = [metadata[file]["lat"] for file in nonroadfiles]
long_list2 = [metadata[file]["lon"] for file in roadfiles]
lat_list2 = [metadata[file]["lat"] for file in roadfiles]
# filename_set = set(filename_list)
df2 = pd.DataFrame(list(zip(long_list2,lat_list2)), columns =['long',"lat"])
df = pd.DataFrame(list(zip(long_list,lat_list)), columns =['long',"lat"])

#initialize an axis
fig, ax = plt.subplots(figsize=(32,32))
# plot map on axis
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

countries.plot(color="lightgrey", ax=ax)
plt.scatter(x = df2["long"], y=df2["lat"],c="red")
plt.scatter(x = df["long"], y=df["lat"],c="blue")

# df2.plot(x="long", y="lat", kind="scatter", 
#          colormap="YlOrRd",c="red", 
#         ax=ax[0])


plt.title('red is roadlocation, blue is nonroad')
plt.savefig("road.png")